# The time to beat is 60-80 seconds per file.

In [1]:

import sys, os
sys.path.insert(0,'/global/homes/b/bpb/metatlas/' )
sys.path.insert(1,'/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages' )
import pandas as pd
from metatlas import metatlas_objects as metob
from metatlas.helpers import dill2plots as dp
import numpy as np
import time
import copy
import dill

('Metatlas live in ', '/global/homes/b/bpb/metatlas/metatlas')
you're running on fb2ee5119e85 at 172.17.0.2 


In [2]:
dp = reload(dp)
groups = dp.select_groups_for_analysis(name = '20160719_RL%',
                                       most_recent = True,
                                       remove_empty = True,
                                       include_list = [])#, exclude_list = ['QC','Blank'])

6
0 20160719_RL_SystemEval_QE144-UMLC_QC 2016-07-19 18:06:45
1 20160719_RL_SystemEval_JGI-1290_QC 2016-07-19 18:06:49
2 20160719_RL_SystemEval_JGI-1290_MeOH 2016-07-19 18:06:55
3 20160719_RL_SystemEval_QE119-1290_MeOH 2016-07-19 18:06:58
4 20160719_RL_SystemEval_QE119-1290_QC 2016-07-19 18:06:59
5 20160719_RL_SystemEval_QE144-UMLC_MeOH 2016-07-19 18:07:02


In [28]:
dp = reload(dp)
groups = dp.select_groups_for_analysis(name = '%Hop%RootExu%neg%',
                                       most_recent = True,
                                       remove_empty = True,
                                       include_list = [])#, exclude_list = ['QC','Blank'])

Reconnecting to database
5
0 Avena_Hopland_KZ_RootExu_control_hilic_neg 2015-11-20 00:15:07
1 Avena_Hopland_KZ_RootExu_3wk_hilic_neg 2015-11-20 00:15:06
2 Avena_Hopland_KZ_RootExu_6wk_hilic_neg 2015-11-20 00:15:06
3 Avena_Hopland_KZ_RootExu_9wk_hilic_neg 2015-11-20 00:15:07
4 Avena_Hopland_KZ_RootExu_12wk_hilic_neg 2015-11-20 00:15:04


In [ ]:
# my_groups = metob.retrieve('Groups',name = '%KZ%HA%pos')
# print my_group.name, len(my_group.items)
# my_file = my_group.items[0]

In [4]:
atlas = dp.get_metatlas_atlas(name='2016719_JG%',do_print = True)

0 11 2016719_JGIQE_QCs_C18_pos 2016-07-19 18:12:07


In [29]:
atlas = dp.get_metatlas_atlas(name='201500826_KZ_Ave_library_qxqct_hilic_neg%',do_print = True)

0 93 201500826_KZ_Ave_library_qxqct_hilic_neg_V2 2016-07-18 20:49:15


In [30]:
myAtlas = atlas[0]
print myAtlas.name
print myAtlas.username

201500826_KZ_Ave_library_qxqct_hilic_neg_V2
bpb


In [31]:
atlas_identifications = dp.export_atlas_to_spreadsheet(myAtlas,'/global/homes/b/bpb/Downloads/KZ_Avena_Exudate_atlases_and groups_1/atlas_201500826_KZ_Ave_library_qxqct_hilic_neg_V2.csv')


# Functions

In [32]:
def df_container_from_metatlas_file(my_file):
    data_df = pd.DataFrame()
    pd_h5_file  = pd.HDFStore(my_file.hdf5_file)
    keys = pd_h5_file.keys()
    pd_h5_file.close()
    df_container = {}
    for k in keys:
        if ('ms' in k) and not ('_mz' in k):
            new_df = pd.read_hdf(my_file.hdf5_file,k)
            df_container[k[1:]] = new_df
    return df_container

def fast_nearest_interp(xi, x, y):
    """Assumes that x is monotonically increasing!!."""
    # Shift x points to centers
    spacing = np.diff(x) / 2
    x = x + np.hstack([spacing, spacing[-1]])
    # Append the last point in y twice for ease of use
    y = np.hstack([y, y[-1]])
    return y[np.searchsorted(x, xi)]

def remove_ms1_data_not_in_atlas(atlas,data):
    things_to_do = [('positive','ms1_pos'),('negative','ms1_neg')]
    for thing in things_to_do:
        if sum(atlas_df.detected_polarity == thing[0])>0:
            atlas_mz = atlas_df[atlas_df.detected_polarity == thing[0]].mz.copy()
            atlas_mz = atlas_mz.sort_values().as_matrix()
            max_mz_tolerance = atlas_df[atlas_df.detected_polarity == thing[0]].mz_tolerance.max()
            if data[thing[1]].shape[0]>1:
                original_mz = data[thing[1]].mz.as_matrix()
                nearest_mz = fast_nearest_interp(original_mz,atlas_mz,atlas_mz)
                data[thing[1]]['ppm_difference'] = abs(original_mz - nearest_mz) / original_mz * 1e6
                query_str = 'ppm_difference < %f'%(max_mz_tolerance)
                data[thing[1]] = data[thing[1]].query(query_str)
    return data

def make_atlas_df(atlas):
    mz = []
    rt = []
    atlas_compound = []
    label = []
    for compound in atlas.compound_identifications:
        if compound.mz_references:
            mz.append(compound.mz_references[0])
        else:
            mz.append(metob.MzReference)
        if compound.rt_references:
            rt.append(compound.rt_references[0])
        else:
            rt.append(metob.RtReference)
        if compound.compound:
            atlas_compound.append(compound.compound[0])
        else:
            atlas_compound.append(metob.Compound())
        label.append(compound.name)
    compound_df = metob.to_dataframe(atlas_compound)
    compound_df.rename(columns = {'name':'compound_name','description':'compound_description'},inplace=True)
    #.rename(columns = {'name':'compound_name'}, inplace = True)
    atlas_df = pd.concat([metob.to_dataframe(rt),metob.to_dataframe(mz), compound_df],axis=1)
    # atlas_df['label'] = label

    atlas_keys = [u'rt_max', u'rt_min', u'rt_peak',u'rt_units', u'detected_polarity', u'mz', u'mz_tolerance',u'mz_tolerance_units']

    # atlas_keys = [u'label','compound_name','compound_description',u'synonyms', u'num_free_radicals', u'number_components', u'permanent_charge', u'rt_max', u'rt_min', u'rt_peak',
    #        u'rt_units', u'detected_polarity', u'mz', u'mz_tolerance',u'mz_tolerance_units',
    #         u'inchi', u'inchi_key', u'neutralized_2d_inchi', u'neutralized_2d_inchi_key', u'neutralized_inchi',
    #        u'neutralized_inchi_key',u'chebi_id', u'hmdb_id', u'img_abc_id', u'kegg_id',u'lipidmaps_id', u'metacyc_id',
    #        u'mono_isotopic_molecular_weight', u'pubchem_compound_id', u'kegg_url', u'chebi_url', u'hmdb_url', u'lipidmaps_url', u'pubchem_url',u'wikipedia_url',  u'source']

    atlas_df = atlas_df[atlas_keys]
    return atlas_df

def get_data_for_mzrt(row,data_df_pos,data_df_neg,extra_time = 0.3,use_mz = 'mz',extra_mz = 0.0):
    min_mz = '(%s >= %5.4f & '%(use_mz,row.mz - row.mz*row.mz_tolerance / 1e6 - extra_mz)
    rt_min = 'rt >= %5.4f & '%(row.rt_min - extra_time)
    rt_max = 'rt <= %5.4f & '%(row.rt_max + extra_time)
    max_mz = '%s <= %5.4f)'%(use_mz,row.mz + row.mz*row.mz_tolerance / 1e6 + extra_mz)
    ms1_query_str = '%s%s%s%s'%(min_mz,rt_min,rt_max,max_mz)
    if row.detected_polarity == 'positive':
        all_df = data_df_pos.query(ms1_query_str)
    else:
        all_df = data_df_neg.query(ms1_query_str)
    return_df = pd.Series({'padded_feature_data':all_df.T.to_dense(),'in_feature':(all_df.rt >= row.rt_min) & (all_df.rt <= row.rt_max)})
    return return_df

def get_ms1_summary(row):
    #A DataFrame of all points typically padded by "extra time"
    all_df = row.padded_feature_data.T
    
    #slice out ms1 data that is NOT padded by extra_time
    ms1_df = all_df[(row.in_feature == True)]#[['i','mz','polarity','rt']]

    num_ms1_datapoints = ms1_df.shape[0]
    if num_ms1_datapoints > 0:
        idx = ms1_df.i.idxmax()
        ms1_peak_df = ms1_df.ix[ms1_df['i'].idxmax()]
        mz_peak = ms1_peak_df.mz
        rt_peak = ms1_peak_df.rt
        mz_centroid = sum(ms1_df.mz * ms1_df.i) / sum(ms1_df.i)
        rt_centroid = sum(ms1_df.rt * ms1_df.i) / sum(ms1_df.i)
        peak_height = ms1_peak_df.i
        peak_area = sum(ms1_df.i)
    else:
        mz_peak = np.nan
        rt_peak = np.nan
        mz_centroid = np.nan
        rt_centroid = np.nan
        peak_height = np.nan
        peak_area = np.nan
        
    return_df = pd.Series({ 'num_ms1_datapoints':num_ms1_datapoints,
                            'mz_peak':mz_peak,
                            'rt_peak':rt_peak,
                            'mz_centroid':mz_centroid,
                            'rt_centroid':rt_centroid,
                            'peak_height':peak_height,
                            'peak_area':peak_area})
    
    return return_df

def get_ms2_data(row):
    #A DataFrame of all points typically padded by "extra time"
    all_df = row.padded_feature_data.T
    
    #slice out ms2 data that is NOT padded by extra_time
    ms2_df = all_df[(row.in_feature == True)]#[['collision_energy','i','mz','polarity','precursor_MZ','precursor_intensity','rt']]

    num_ms2_datapoints = ms2_df.shape[0]
        
    return_df = pd.Series({'ms2_datapoints':ms2_df.T.to_dense(),
                            'num_ms2_datapoints':num_ms2_datapoints})
    return return_df


def prefilter_ms1_dataframe_with_boundaries(data_df,rt_max,rt_min,mz_min,mz_max,extra_time = 1, extra_mz = 1):
    if data_df.shape[0]==0:
        return []
    prefilter_query_str = 'rt < %5.4f & rt > %5.4f & mz > %5.4f & mz < %5.4f'%(rt_max+extra_time,rt_min-extra_time,mz_min - extra_mz, mz_max+extra_mz)
    new_df = data_df.query(prefilter_query_str)
    return new_df

def get_ms1_eic(row):
    #A DataFrame of all points typically padded by "extra time"
    all_df = row.padded_feature_data.T
    ms1_df = all_df[['i','mz','rt']]
    ms1_df = ms1_df.sort_values('rt',ascending=True)
    return pd.Series({'eic':ms1_df.T.to_dense()})


def retrieve_most_intense_msms_scan(data):
    import numpy as np
    urt,idx = np.unique(data['rt'],return_index=True)
    sx = np.argsort(data['precursor_intensity'][idx])[::-1]
    prt = data['rt'][idx[sx]]
    pmz = data['precursor_MZ'][idx[sx]]
    pintensity = data['precursor_intensity'][idx[sx]]
    #setup data format for searching
    msms_data = {}
    msms_data['spectra'] = []
    msms_data['precursor_mz'] = []
    msms_data['precursor_intensity'] = []
    idx = np.argwhere((data['precursor_MZ'] == pmz[0]) & (data['rt'] == prt[0] )).flatten()
    arr = np.array([data['mz'][idx], data['i'][idx]]).T
    msms_data['spectra'] = arr
    msms_data['precursor_mz'] = pmz
    msms_data['precursor_intensity'] = pintensity
    return msms_data

def get_data_for_atlas_and_lcmsrun(atlas,df_container):
    filtered_ms1_pos = prefilter_ms1_dataframe_with_boundaries(df_container['ms1_pos'],
                                                               atlas_df[atlas_df.detected_polarity == 'positive'].rt_max.max(),
                                                               atlas_df[atlas_df.detected_polarity == 'positive'].rt_min.min(),
                                                               atlas_df[atlas_df.detected_polarity == 'positive'].mz.min(),
                                                               atlas_df[atlas_df.detected_polarity == 'positive'].mz.max())
    filtered_ms1_neg = prefilter_ms1_dataframe_with_boundaries(df_container['ms1_neg'],
                                                           atlas_df[atlas_df.detected_polarity == 'negative'].rt_max.max(),
                                                           atlas_df[atlas_df.detected_polarity == 'negative'].rt_min.min(),
                                                           atlas_df[atlas_df.detected_polarity == 'negative'].mz.min(),
                                                           atlas_df[atlas_df.detected_polarity == 'negative'].mz.max())

    #here it needs to use the atlas.polarity to know which df to look in
    ms1_feature_data = atlas_df.apply(lambda x: get_data_for_mzrt(x,filtered_ms1_pos,filtered_ms1_neg),axis=1)
    ms1_summary = ms1_feature_data.apply(get_ms1_summary,axis=1)
    ms1_eic = ms1_feature_data.apply(get_ms1_eic,axis=1)

    #here it needs to use the atlas.polarity to know which df to look in
    
    filtered_ms2_pos = prefilter_ms1_dataframe_with_boundaries(df_container['ms2_pos'],
                                                           atlas_df[atlas_df.detected_polarity == 'positive'].rt_max.max(),
                                                           atlas_df[atlas_df.detected_polarity == 'positive'].rt_min.min(),
                                                           0,
                                                           atlas_df[atlas_df.detected_polarity == 'positive'].mz.max())
    filtered_ms2_neg = prefilter_ms1_dataframe_with_boundaries(df_container['ms2_neg'],
                                                           atlas_df[atlas_df.detected_polarity == 'negative'].rt_max.max(),
                                                           atlas_df[atlas_df.detected_polarity == 'negative'].rt_min.min(),
                                                           0,
                                                           atlas_df[atlas_df.detected_polarity == 'negative'].mz.max())
    ms2_feature_data = atlas_df.apply(lambda x: get_data_for_mzrt(x,filtered_ms2_pos,filtered_ms2_neg,use_mz = 'precursor_MZ',extra_mz = 0.01),axis=1)
    ms2_data = ms2_feature_data.apply(get_ms2_data,axis=1)
    dict_ms1_summary = [dict(row) for i,row in ms1_summary.iterrows()]
    dict_eic = [row.eic.T.to_dict(orient='list') for i,row in ms1_eic.iterrows()]
    for i,d in enumerate(dict_eic):
        dict_eic[i]['intensity'] = dict_eic[i].pop('i')
    dict_ms2 = [row.ms2_datapoints.T.to_dict(orient='list') for i,row in ms2_data.iterrows()]
    return dict_ms1_summary,dict_eic,dict_ms2


In [12]:
#These atlas/file throw an error because they don't have MSMS data
# 20160715_SK_SystemEval_QE119-1290_C18-LS-B15180-1to2_PosMS1_InjSK-MeOH_r01_s01rerun.mzML
# 2016719_JGIQE_QCs_C18_pos

#get_data_for_atlas_and_lcmsrun(atlas,df_container)
#has many steps that shouldn't be done if there is no data.

# dict_ms1_summary,dict_eic,dict_ms2 = get_data_for_atlas_and_lcmsrun(atlas_df,df_container)
print my_file.name
print myAtlas.name

20160715_SK_SystemEval_QE119-1290_C18-LS-B15180-1to2_PosMS1_InjSK-MeOH_r01_s01rerun.mzML
2016719_JGIQE_QCs_C18_pos


In [33]:
output_filename = '/global/homes/b/bpb/Downloads/KZ_Avena_Exudate_atlases_and groups_1/neg_data.pkl'
atlas_df = make_atlas_df(myAtlas)
data = []
for my_group in groups:
    for my_file in my_group.items:
        t0 = time.time()
        df_container = df_container_from_metatlas_file(my_file)
        df_container = remove_ms1_data_not_in_atlas(atlas_df,df_container)

        dict_ms1_summary,dict_eic,dict_ms2 = get_data_for_atlas_and_lcmsrun(atlas_df,df_container)
        row = []
        for i in range(len(dict_eic)):
            result = {}
            result['atlas_name'] = myAtlas.name
            result['atlas_unique_id'] = myAtlas.unique_id
            result['lcmsrun'] = my_file
            result['group'] = my_group
            temp_compound = copy.deepcopy(myAtlas.compound_identifications[i])
            result['identification'] = temp_compound
            result['data'] = {}
            result['data']['eic'] = dict_eic[i]
            result['data']['ms1_summary'] = dict_ms1_summary[i]
            result['data']['msms'] = {}
            if dict_ms2[i]['mz']:
                for k in dict_ms2[0].keys():
                    dict_ms2[i][k] = np.asarray(dict_ms2[i][k])
#                 if temp_compound.mz_references[0].observed_polarity == 'positive':
#                     dict_ms2[i]['polarity'] = dict_ms2[i]['mz'] * 0.0 + 1.0
#                 else:
#                     dict_ms2[i]['polarity'] = dict_ms2[i]['mz'] * 0.0
                result['data']['msms']['data'] = dict_ms2[i]
            else:
                result['data']['msms']['data'] = []
            row.append(result)
        data.append(row)
        print my_file.name,time.time() - t0
with open(output_filename,'w') as f:
    dill.dump(data,f)

20151016_pHILIC___NEG_MSMS_KZ_RootExuControl__1_of_2_MeOHExt__Run9.mzML 3.60179805756
20151016_pHILIC___NEG_MSMS_KZ_RootExuControl__2_of_2_MeOHExt__Run43.mzML 3.12154388428
20151016_pHILIC___NEG_MSMS_KZ_RootExu_3wk_1_of_4_MeOHExt__Run11.mzML 3.34261989594
20151016_pHILIC___NEG_MSMS_KZ_RootExu_3wk_2_of_4_MeOHExt__Run54.mzML 3.25781297684
20151016_pHILIC___NEG_MSMS_KZ_RootExu_3wk_3_of_4_MeOHExt__Run36.mzML 3.52949810028
20151016_pHILIC___NEG_MSMS_KZ_RootExu_3wk_4_of_4_MeOHExt__Run27.mzML 3.0998711586
20151016_pHILIC___NEG_MSMS_KZ_RootExu_6wk_1_of_4_MeOHExt__Run14.mzML 3.37020993233
20151016_pHILIC___NEG_MSMS_KZ_RootExu_6wk_2_of_4_MeOHExt__Run48.mzML 3.55708003044
20151016_pHILIC___NEG_MSMS_KZ_RootExu_6wk_3_of_4_MeOHExt__Run33.mzML 3.4470539093
20151016_pHILIC___NEG_MSMS_KZ_RootExu_6wk_4_of_4_MeOHExt__Run57.mzML 3.04358410835
20151016_pHILIC___NEG_MSMS_KZ_RootExu_9wk_1_of_4_MeOHExt__Run17.mzML 3.21957302094
20151016_pHILIC___NEG_MSMS_KZ_RootExu_9wk_2_of_4_MeOHExt__Run51.mzML 3.59547400475

In [ ]:
# result = {}
# result['atlas_name'] = myAtlas.name
# result['atlas_unique_id'] = myAtlas.unique_id
# result['lcmsrun'] = treatment_groups.items[j]
# result['group'] = treatment_groups
# temp_compound = copy.deepcopy(compound)
# result['identification'] = temp_compound

# result['data'] = return_data...
# return_data['ms1_summary']['polarity'] = polarity
# return_data['ms1_summary']['mz_centroid'] = np.sum(np.multiply(ms1_data['i'],ms1_data['mz'])) / np.sum(ms1_data['i'])
# return_data['ms1_summary']['rt_centroid'] = np.sum(np.multiply(ms1_data['i'],ms1_data['rt'])) / np.sum(ms1_data['i'])
# idx = np.argmax(ms1_data['i'])
# return_data['ms1_summary']['mz_peak'] = ms1_data['mz'][idx]
# return_data['ms1_summary']['rt_peak'] = ms1_data['rt'][idx]        
# return_data['ms1_summary']['peak_height'] = ms1_data['i'][idx]
# return_data['ms1_summary']['peak_area'] = np.sum(ms1_data['i'])

# return_data['msms']['most_intense_precursor'] = retrieve_most_intense_msms_scan(fragmentation_data)
# return_data['msms']['data'] = fragmentation_data
# return_data['msms']['polarity'] = polarity

# return_data['eic']['rt'] = rt
# return_data['eic']['intensity'] = intensity
# return_data['eic']['polarity'] = polarity

In [ ]:
# pd.concat([atlas_df,temp,temp2],axis=1).head()

In [ ]:
def get_data_for_groups_and_atlas(group,myAtlas,output_filename,use_set1 = False):
    """
    get and pickle everything This is MSMS, raw MS1 datapoints, compound, group info, and file info
    """
    data = []
    import copy as copy
    for i,treatment_groups in enumerate(group):
        for j in range(len(treatment_groups.items)):
            myFile = treatment_groups.items[j].hdf5_file
    #         try:
    #             rt_reference_index = int(treatment_groups.name[-1]) - 1
    #         except:
    #             rt_reference_index = 3
            print i,len(group),myFile
            row = []
            for compound in myAtlas.compound_identifications:
                result = {}
                result['atlas_name'] = myAtlas.name
                result['atlas_unique_id'] = myAtlas.unique_id
                result['lcmsrun'] = treatment_groups.items[j]
                result['group'] = treatment_groups
                temp_compound = copy.deepcopy(compound)
                if use_set1:
                    if '_Set1' in treatment_groups.name:
                        temp_compound.rt_references[0].rt_min -= 0.2
                        temp_compound.rt_references[0].rt_max -= 0.2
                        temp_compound.rt_references[0].rt_peak -= 0.2
                    temp_compound.mz_references[0].mz_tolerance = 20
                result['identification'] = temp_compound
                result['data'] = ma_data.get_data_for_a_compound(temp_compound.mz_references[0],
                                        temp_compound.rt_references[0],
                                        [ 'ms1_summary', 'eic', 'msms' ],
                                        myFile,0.2)
    #                 print result['data']['ms1_summary']
                row.append(result)
            data.append(row)
        with open(output_filename,'w') as f:
            dill.dump(data,f)
import numpy as np
import os.path
import sys
import tables
def get_unique_scan_data(data):
    """
    Input:
    data - numpy nd array containing MSMS data
    
    Output:
    rt - retention time of scan
    pmz - precursor m/z of scan
    Both are sorted by descending precursor ion intensity
    
    for data returned from h5query.get_data(),
    return the retention time and precursor m/z
    sorted by descending precursor ion intensity
    """
    urt,idx = np.unique(data['rt'],return_index=True)
    sx = np.argsort(data['precursor_intensity'][idx])[::-1]
    prt = data['rt'][idx[sx]]
    pmz = data['precursor_MZ'][idx[sx]]
    pintensity = data['precursor_intensity'][idx[sx]]
    return prt,pmz,pintensity


def get_non_redundant_precursor_list(prt,pmz,rt_cutoff,mz_cutoff):
    """
    Input:
    rt - retention time of scan
    pmz - precursor m/z of scan
    Both are sorted by descending precursor ion intensity
    rt_cutoff - 
    mz_cutoff - 
    
    Output:
    list_of_prt - list of 
    list_of_pmz - list of 
    """
    
    list_of_pmz = [] #contains list of precursor m/z [pmz1,pmz2,...,pmz_n]
    list_of_prt = [] #contains list of precursor rt [prt1,prt2,...,prt_n]
    
    for i in range(len(prt)):
        if len(list_of_pmz) == 0:
            # none are in the list yet; so there is nothing to check
            list_of_pmz.append(pmz[i])
            list_of_prt.append(prt[i])
        else:
            # check if new rt qualifies for inclusion
            if min(abs(list_of_prt - prt[i])) > rt_cutoff or min(abs(list_of_pmz - pmz[i])) > mz_cutoff:
                list_of_pmz.append(pmz[i])
                list_of_prt.append(prt[i])
    return list_of_prt,list_of_pmz


def organize_msms_scan_data(data,list_of_prt,list_of_pmz,list_of_pintensity):
    #setup data format for searching
    msms_data = {}
    msms_data['spectra'] = []
    msms_data['precursor_mz'] = []
    msms_data['precursor_rt'] = []
    msms_data['precursor_intensity'] = []
    for i,(prt,pmz,pintensity) in enumerate(zip(list_of_prt,list_of_pmz,list_of_pintensity)):
        idx = np.argwhere((data['precursor_MZ'] == pmz) & (data['rt'] == prt )).flatten()
        arr = np.array([data['mz'][idx], data['i'][idx]]).T
        msms_data['spectra'].append(arr)
        msms_data['precursor_mz'].append(pmz)
        msms_data['precursor_rt'].append(prt)
        msms_data['precursor_intensity'].append(pintensity)
    return msms_data

def retrieve_most_intense_msms_scan(data):
    import numpy as np
    urt,idx = np.unique(data['rt'],return_index=True)
    sx = np.argsort(data['precursor_intensity'][idx])[::-1]
    prt = data['rt'][idx[sx]]
    pmz = data['precursor_MZ'][idx[sx]]
    pintensity = data['precursor_intensity'][idx[sx]]
    #setup data format for searching
    msms_data = {}
    msms_data['spectra'] = []
    msms_data['precursor_mz'] = []
    msms_data['precursor_intensity'] = []
    idx = np.argwhere((data['precursor_MZ'] == pmz[0]) & (data['rt'] == prt[0] )).flatten()
    arr = np.array([data['mz'][idx], data['i'][idx]]).T
    msms_data['spectra'] = arr
    msms_data['precursor_mz'] = pmz
    msms_data['precursor_intensity'] = pintensity
    return msms_data


def get_data_for_a_compound(mz_ref,rt_ref,what_to_get,h5file,extra_time):
    """
    A helper function to query the various metatlas data selection 
    commands for a compound defined in an experimental atlas.

    Parameters
    ----------
    MZref : a MetAtlas Object for a m/z reference Class
        this contains the m/z, m/z tolerance, and tolerance units to slice the m/z dimension
    RTref : a MetAtlas Object for a retention time reference Class
        this contains the rt min, max, peak, and units to slice the retention time dimension
    what_to_get : a list of strings
        this contains one or more of [ 'ms1_summary', 'eic', '2dhist', 'msms' ]
    h5_file : str
        Path to input_file
    polarity : int
        [0 or 1] for negative or positive ionzation
    
    Returns
    -------
    """
    #TODO : polarity should be handled in the experiment and not a loose parameter
    import numpy as np
    from metatlas import h5_query as h5q
    import tables
    
    #get a pointer to the hdf5 file
    fid = tables.open_file(h5file) #TODO: should be a "with open:"

    if mz_ref.detected_polarity  == 'positive':
        polarity = 1
    else:
        polarity = 0
        
    
    mz_theor = mz_ref.mz
    if mz_ref.mz_tolerance_units  == 'ppm': #convert to ppm
        ppm_uncertainty = mz_ref.mz_tolerance
    else:
        ppm_uncertainty = mz_ref.mz_tolerance / mz_ref.mz * 1e6
        
#     if 'min' in rt_ref.rt_units: #convert to seconds
#     rt_min = rt_ref.rt_min / 60
#     rt_max = rt_ref.rt_max / 60
#     else:
    rt_min = rt_ref.rt_min
    rt_max = rt_ref.rt_max
        
    mz_min = mz_theor - mz_theor * ppm_uncertainty / 1e6
    mz_max = mz_theor + mz_theor * ppm_uncertainty / 1e6
    
    return_data = {}
    
    if 'ms1_summary' in what_to_get:
        #Get Summary Data
        
        #First get MS1 Raw Data
        ms_level=1
        return_data['ms1_summary'] = {}
        try:
            ms1_data = h5q.get_data(fid, 
                                     ms_level=1,
                                     polarity=polarity,
                                     min_mz=mz_min,
                                     max_mz=mz_max,
                                     min_rt=rt_min,
                                     max_rt=rt_max)

            return_data['ms1_summary']['polarity'] = polarity
            return_data['ms1_summary']['mz_centroid'] = np.sum(np.multiply(ms1_data['i'],ms1_data['mz'])) / np.sum(ms1_data['i'])
            return_data['ms1_summary']['rt_centroid'] = np.sum(np.multiply(ms1_data['i'],ms1_data['rt'])) / np.sum(ms1_data['i'])
            idx = np.argmax(ms1_data['i'])
            return_data['ms1_summary']['mz_peak'] = ms1_data['mz'][idx]
            return_data['ms1_summary']['rt_peak'] = ms1_data['rt'][idx]        
            return_data['ms1_summary']['peak_height'] = ms1_data['i'][idx]
            return_data['ms1_summary']['peak_area'] = np.sum(ms1_data['i'])

        except:
            return_data['ms1_summary']['polarity'] = []
            return_data['ms1_summary']['mz_centroid'] = []
            return_data['ms1_summary']['rt_centroid'] = []
            return_data['ms1_summary']['mz_peak'] = []
            return_data['ms1_summary']['rt_peak'] = []
            return_data['ms1_summary']['peak_height'] = []
            return_data['ms1_summary']['peak_area'] = []

    
    if 'eic' in what_to_get:
        #Get Extracted Ion Chromatogram
        # TODO : If a person calls for summary, then they will already have the MS1 raw data
        return_data['eic'] = {}
        try:
            rt,intensity = h5q.get_chromatogram(fid, mz_min, mz_max, ms_level=ms_level, polarity=polarity, min_rt = rt_min - extra_time, max_rt = rt_max + extra_time)
            return_data['eic']['rt'] = rt
            return_data['eic']['intensity'] = intensity
            return_data['eic']['polarity'] = polarity

        except:    
            return_data['eic']['rt'] = []
            return_data['eic']['intensity'] = []
            return_data['eic']['polarity'] = []
    
    if '2dhist' in what_to_get:
        #Get 2D histogram of intensity values in m/z and retention time
        mzEdges = np.logspace(np.log10(100),np.log10(1000),10000)
#         mzEdges = np.linspace(mz_theor - 3, mz_theor + 30,100) #TODO : number of mz bins should be an optional parameter
        rtEdges = np.linspace(rt_min,rt_max,100) #TODO : number of rt bins should be an optional parameter. When not provided, it shoulddefauly to unique bins
        ms_level = 1 #TODO : ms_level should be a parameter
        return_data['2dhist'] = {}
        return_data['2dhist'] = h5q.get_heatmap(fid,mzEdges,rtEdges,ms_level,polarity)
        return_data['2dhist']['polarity'] = polarity
    
    if 'msms' in what_to_get:
        #Get Fragmentation Data
        ms_level=2
        return_data['msms'] = {}
        try:
            fragmentation_data = h5q.get_data(fid, 
                                     ms_level=ms_level,
                                     polarity=polarity,
                                     min_mz=0,
                                     max_mz=mz_theor+2,#TODO : this needs to be a parameter
                                     min_rt=rt_min,
                                     max_rt=rt_max,
                                     min_precursor_MZ=mz_min - 0.015,
                                     max_precursor_MZ=mz_max + 0.015) #Add the 0.01 because Thermo doesn't store accurate precursor m/z
        #                     min_precursor_intensity=0, #TODO : this needs to be a parameter
        #                     max_precursor_intensity=0,#TODO : this needs to be a parameter
        #                     min_collision_energy=0,#TODO : this needs to be a parameter
        #                     max_collision_energy=0)#TODO : this needs to be a parameter
    #         prt,pmz = get_unique_scan_data(fragmentation_data)
    #         rt_cutoff = 0.23
    #         mz_cutoff = 0.05
    #         list_of_prt,list_of_pmz = get_non_redundant_precursor_list(prt,pmz,rt_cutoff,mz_cutoff)
    #         return_data['msms']['data'] = organize_msms_scan_data(fragmentation_data,list_of_prt,list_or_pmz)
            return_data['msms']['most_intense_precursor'] = retrieve_most_intense_msms_scan(fragmentation_data)
            return_data['msms']['data'] = fragmentation_data
            return_data['msms']['polarity'] = polarity
        except:
            return_data['msms']['most_intense_precursor'] = []
            return_data['msms']['data'] = []
            return_data['msms']['polarity'] = []
            
    fid.close() #close the file
    return return_data



def get_dill_data(fname):
    """
    Parameters
    ----------
    fname: dill file name

    Returns a list containing the data present in the dill file
    -------
    """
    import dill

    data = list()
    
    if os.path.exists(fname):
        with open(fname,'r') as f:
            try:
                data = dill.load(f)
            except IOError as e:
                print "I/O error({0}): {1}".format(e.errno, e.strerror)
            except:  # handle other exceptions such as attribute errors
                print "Unexpected error:", sys.exc_info()[0]


    return data


def get_group_names(data):
    """
    Parameters
    ----------
    data: either a file name (str) or a list generated from loading the dill file

    Returns list containing the group names present in the dill file
    -------
    """

    # if data is a string then it's a file name - get its data
    if isinstance(data, basestring):
        data = get_dill_data(data)

    group_names = list()
    for i,d in enumerate(data):
        group_names.append(d[0]['group'].name)

    return group_names


def get_file_names(data):
    """
    Parameters
    ----------
    data: either a file name (str) or a list generated from loading the dill file

    Returns list containing the hdf file names present in the dill file
    -------
    """
    import os.path

    # if data is a string then it's a file name - get its data
    if isinstance(data, basestring):
        data = get_dill_data(data)

    file_names = list()
    for i,d in enumerate(data):
        file_names.append(os.path.basename(d[0]['lcmsrun'].hdf5_file))

    return file_names


def get_compound_names(data):
    """
    Parameters
    ----------
    data: either a file name (str) or a list generated from loading the dill file

    Returns a tuple of lists containing the compound names and compound objects present in the dill file
    -------
    """
    from collections import defaultdict
    import re

    # if data is a string then it's a file name - get its data
    if isinstance(data, basestring):
        data = get_dill_data(data)

    compound_names = list()
    compound_objects = list()

    for i,d in enumerate(data[0]):
        compound_objects.append(d['identification'])
        if len(d['identification'].compound) > 0:
            _str = d['identification'].compound[0].name
        else:
            _str = d['identification'].name
        _str = _str.split('///')[0]
        newstr = '%s_%s_%s_%5.2f'%(_str,d['identification'].mz_references[0].detected_polarity,
                d['identification'].mz_references[0].adduct,d['identification'].rt_references[0].rt_peak)
        newstr = re.sub('\.', 'p', newstr) #2 or more in regexp

        newstr = re.sub('[\[\]]','',newstr)
        newstr = re.sub('[^A-Za-z0-9+-]+', '_', newstr)
        newstr = re.sub('i_[A-Za-z]+_i_', '', newstr)
        if newstr[0] == '_':
            newstr = newstr[1:]
        if newstr[0] == '-':
            newstr = newstr[1:]
        if newstr[-1] == '_':
            newstr = newstr[:-1]

        newstr = re.sub('[^A-Za-z0-9]{2,}', '', newstr) #2 or more in regexp
        compound_names.append(newstr)

    # If duplicate compound names exist, then append them with a number
    D = defaultdict(list)
    for i,item in enumerate(compound_names):
        D[item].append(i)
    D = {k:v for k,v in D.items() if len(v)>1}
    for k in D.keys():
        for i,f in enumerate(D[k]):
            compound_names[f] = '%s%d'%(compound_names[f],i)

    return (compound_names, compound_objects)
